# 第五讲 pandas入门

* pandas 是基于NumPy 的一种工具，该工具是为解决数据分析任务而创建的。

* pandas最初被作为金融数据分析工具而开发出来，因此，pandas为时间序列分析提供了很好的支持。 

* Pandas的名称来自于面板数据（panel data）和python数据分析（data analysis）。

#### pandas vs NumPy

* pandas支持大部分NumPy风格的数组计算

* NumPy更适合处理同质型的的数值类数组数据,而pandas使用用来处理表格型或异质型数据

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
from pandas import Series, DataFrame

## 1. pandas数据结构介绍

* 为了入门pandas，需要熟悉两个常用的工具数据结构：Series和DataFrame。

###  （1）Series

* Series是一种一维的数组型对象，它包含了一个值序列（与NumPy中的类型相似），并且包含了数据标签，称为索引（index）。


* 最简单的序列可以仅仅由一个数组形成：

In [124]:
obj = pd.Series([4, 7, 'str1', 3])
obj

0       4
1       7
2    str1
3       3
dtype: object

In [33]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

* 交互式环境中Series的字符串表示，索引在左边，值在右边。

In [125]:
obj.values

array([4, 7, 'str1', 3], dtype=object)

In [126]:
obj.index  # like range(4)

RangeIndex(start=0, stop=4, step=1)

* 通常需要创建一个索引序列，用标签标识每个数据点：

In [10]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [128]:
obj2.values

array([-5.3,  7.2,  3.6,  4.5,  nan])

In [127]:
obj2.index

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

* 与NumPy的数组相比，可以在从数据中选择数据的时候使用标签来进行索引：

In [12]:
obj2['a']

-5

In [13]:
obj2['d'] = 6

In [14]:
obj2[['c', 'a', 'd']]

c    3
a   -5
d    6
dtype: int64

In [130]:
obj2[2]

3.6

In [131]:
obj2[:2]

a   -5.3
b    7.2
dtype: float64

In [132]:
obj2[2:]

c    3.6
d    4.5
e    NaN
dtype: float64

* 使用NumPy的函数或NumPy风格的操作，比如使用布尔值数组进行过滤，与标量相乘，或是应用数学函数，这些操作将保存索引值连接：

In [15]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [16]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [17]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

* 从另一个角度考虑Series，可以认为它是一个长度固定且有序的字典，因为它将索引值和数据值按位置配对。

In [18]:
'b' in obj2

True

In [19]:
'e' in obj2

False

* 如果已经有数据包含在Python字典中，可以使用字典生成一个Series：

In [20]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

* 当把字典传递给Series构造函数时，产生的Series的索引将是排序好的字典键。

In [134]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

* pandas中使用 <b>isnull</b>和<b>notnull</b>函数来检查缺失数据：

In [135]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [136]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [137]:
obj4[pd.notnull(obj4)]

Ohio      35000.0
Oregon    16000.0
Texas     71000.0
dtype: float64

* <b>isnull</b>和<b>notnull</b>也是Series的实例方法：

In [24]:
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [139]:
obj4[obj4.notnull()]

Ohio      35000.0
Oregon    16000.0
Texas     71000.0
dtype: float64

* 对于很多应用来说，在数学操作中自动对齐索引是Series的一个非常有用的特性：

In [26]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [27]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [28]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

* Series对象自身和其索引都有name属性，这个特性与pandas其他重要功能集成在一起：

In [24]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

* Series的索引可以通过按位置赋值的方式进行改变：

In [25]:
obj
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### （2）DataFrame

* DataFrame表示的是矩阵的数据表，它包含已排序的列集合，每一列可以是不同的值类型（数值、字符串、布尔值等）。

In [29]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

* 产生的DataFrame会自动为Series分配索引，并且列会按照排序的顺序排列：

In [28]:
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


* 对于大型DataFrame，head方法将会只选出头部的五行：

In [29]:
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


* 如果指定了列的顺序，DataFrame的列将会按照指定顺序排列：

In [30]:
pd.DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


* 如果传的列不包含在字典中，将会在结果中出现缺失值：

In [141]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                      index=['one', 'two', 'three', 'four',
                             'five', 'six'])
frame2

,year,state,pop,debt
one,NaN,NaN,NaN,NaN
two,NaN,NaN,NaN,NaN
three,NaN,NaN,NaN,NaN
four,NaN,NaN,NaN,NaN
five,NaN,NaN,NaN,NaN
six,NaN,NaN,NaN,NaN


In [142]:

frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

* DataFrame中的一列，可以按字典型标记或属性那样检索为Series：

In [143]:
frame2['state']

one     NaN
two     NaN
three   NaN
four    NaN
five    NaN
six     NaN
Name: state, dtype: float64

In [144]:
frame2.year

one     NaN
two     NaN
three   NaN
four    NaN
five    NaN
six     NaN
Name: year, dtype: float64

* 注意，返回的Series与原DataFrame有相同的索引，且其name属性也会被合理地设置。

* 行也可以通过位置或特殊属性<b>loc</b>进行选取：

In [35]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

* 列的引用是可以修改的。例如，空的'debt'列可以赋值为标量值或值数组：

In [36]:
frame2['debt'] = 16.5
frame2
frame2['debt'] = np.arange(6.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,3.2,5.0


* 将列表或数组赋值给一个列时，值的长度必须和DataFrame的长度相匹配。

In [37]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


* 如果被赋值的列并不存在，则会生成一个新的列。<b>del</b>关键字可以像在字典中那样对DataFrame删除列。

* 在<b>del</b>的例子中，首先增加一列，这一列是布尔值，判断条件是state列是否为'Ohio'：

In [38]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


* <b>del</b>方法可以用于移除之前新建的列：

In [39]:
del frame2['eastern']
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

* 另一种常用的数据形式是包含字典的嵌套字典：

In [40]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

* 如果嵌套字典被赋值给DataFrame，pandas会将字典的键作为列，将内部字典的键作为行索引：

In [41]:
frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


* 可以使用类似NumPy的语法对DataFrame进行转置操作（调换行和列）：

In [42]:
frame3.T

,2001,2002,2000
Nevada,2.4,2.9,NaN
Ohio,1.7,3.6,1.5


* 内部字典的键被联合、排序后形成了结果的索引。如果明确指定了索引，内部字典的键将不会被排序：

In [43]:
pd.DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


* 包含Series的字典也可以用于构造DataFrame：

In [44]:
pdata = {'Ohio': frame3['Ohio'][:-1],
         'Nevada': frame3['Nevada'][:2]}
pd.DataFrame(pdata)

,Ohio,Nevada
2001,1.7,2.4
2002,3.6,2.9


* 如果设置了DataFrame的index和columns的name属性，则这些name属性也会被显示出来：

In [45]:
frame3.index.name = 'year'; frame3.columns.name = 'state'
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


* 跟Series一样，DataFrame的values属性也会以二维ndarray的形式返回DataFrame中的数据：

In [46]:
frame3.values

array([[2.4, 1.7],
       [2.9, 3.6],
       [nan, 1.5]])

* 如果DataFrame各列的数据类型不同，则values的dtype就会选用能兼容所有列的数据类型：

In [48]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

### （3）索引对象

* pandas的索引对象是用于存储轴标签和其他元数据（比如轴名称或标签）。构建Series或DataFrame时，所用到的任何数组或其他序列的标签都会被转换成一个索引对象：

In [49]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index
index[1:]

Index(['b', 'c'], dtype='object')

* 索引对象是不可变的，因此用户不能对其进行修改：

index[1] = 'd'  # TypeError

In [50]:
labels = pd.Index(np.arange(3))
labels
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2
obj2.index is labels

True

* 除了类似于数组，索引对象也类似一个固定大小的集合：

In [51]:
frame3
frame3.columns
'Ohio' in frame3.columns
2003 in frame3.index

False

* 与python的集合不同，pandas的Index可以包含重复的标签：

In [52]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

## 2. 基本功能

* Series和DataFrame中数据交互的基础机制。

### （1） 重建索引

* pandas对象的一个重要方法是reindex，其作用是创建一个新对象，它的数据符合新的索引

In [80]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [81]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [82]:
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

* 在重建索引时进行插值操作

In [83]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [85]:
obj4 = obj3.reindex(range(6), method='ffill')
obj4

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

* 借助DataFrame，reindex可以修改行索引和列索引（也可以同时改变二者）。只传递一个序列时，会重新索引结果的行：

In [86]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [87]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


* 列可以用columns关键字重新索引：

In [92]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [93]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


### （2）轴向上删除条目

* 在轴向上删除一个或多个条目很简单，只要有一个索引数组或列表即可。由于需要执行一些数据整理和集合逻辑，所以drop方法返回的是一个在指定轴上删除了指定值的新对象：

In [95]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [96]:
new_obj = obj.drop('c')
new_obj


a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [97]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

In [98]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [99]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [100]:
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [101]:
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


In [103]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


* inplace 参数

In [104]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [105]:
obj.drop('a', inplace=False)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

### （3）索引、选择与过滤

* Series索引（obj[...]）的工作方式类似于NumPy数组的索引，只不过Series的索引值不只是整数。下面是几个例子：

In [106]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj


a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [107]:
obj['b']


1.0

In [108]:
obj[1]


1.0

In [109]:
obj[2:4]


c    2.0
d    3.0
dtype: float64

In [110]:
obj[['b', 'a', 'd']]


b    1.0
a    0.0
d    3.0
dtype: float64

In [111]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [112]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

In [113]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

In [114]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

* 使用单个值或序列，可以从DataFrame中索引数一个多个列

In [115]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [116]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [117]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [118]:
data[:2]


,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [119]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [120]:
data < 5


,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [121]:
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


#### Selection with loc and iloc

In [75]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int32

In [76]:
data.iloc[2, [3, 0, 1]]
data.iloc[2]
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


In [77]:
data.loc[:'Utah', 'two']
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


### （4） 整数索引

* 处理整数索引的pandas对象常常难住新手，因为它与Python内置的列表和元组的索引语法不同。例如，你可能不认为下面的代码会出错：

In [148]:
ser = pd.Series(np.arange(3.))
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [149]:
ser[-1]

KeyError: -1

* 这里，pandas可以勉强进行整数索引，但是会导致小bug。我们有包含0,1,2的索引，但是引入用户想要的东西（基于标签或位置的索引）很难：

* 另一方面，对于非整数索引，则不会有潜在的歧义：

In [152]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2

a    0.0
b    1.0
c    2.0
dtype: float64

In [153]:
ser2[-1]

2.0

* 为了进行统一，如果轴索引含有整数，数据选取总会使用标签。为了更准确，请使用loc（标签）或iloc（整数）：

In [154]:
ser[:1]


0    0.0
dtype: float64

In [155]:
ser.loc[:1]


0    0.0
1    1.0
dtype: float64

In [156]:
ser.iloc[:1]

0    0.0
dtype: float64

### （5）算术和数据对齐 

* pandas最重要的一个功能是，它可以对不同索引的对象进行算术运算。在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集。对于有数据库经验的用户，这就像在索引标签上进行自动外连接。看一个简单的例子：

In [83]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])
s1
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

* 将它们相加就会产生：

In [84]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In [ ]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df1
df2

In [ ]:
df1 + df2

In [ ]:
df1 = pd.DataFrame({'A': [1, 2]})
df2 = pd.DataFrame({'B': [3, 4]})
df1
df2
df1 - df2

#### ① 使用填充值的算术方法 

In [ ]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
                   columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
                   columns=list('abcde'))
df2.loc[1, 'b'] = np.nan
df1
df2

In [ ]:
df1 + df2

In [ ]:
df1.add(df2, fill_value=0)

In [ ]:
1 / df1
df1.rdiv(1)

In [ ]:
df1.reindex(columns=df2.columns, fill_value=0)

#### ② DataFrame和Series之间的操作

In [ ]:
arr = np.arange(12.).reshape((3, 4))
arr
arr[0]
arr - arr[0]

In [ ]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.iloc[0]
frame
series

In [ ]:
frame - series

In [ ]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame + series2

In [ ]:
series3 = frame['d']
frame
series3
frame.sub(series3, axis='index')

### （6）函数应用和映射

* NumPy的通用函数（逐元素数组方法）对pandas对象也有效：

In [85]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame
np.abs(frame)

,b,d,e
Utah,1.494199,0.405303,0.308872
Ohio,1.025365,1.194160,1.263462
Texas,0.670473,1.154328,0.768261
Oregon,0.783044,3.831244,0.205178


* 另一个常用的操作是将函数应用到一行或一列的一维数组上。DataFrame的apply方法可以实现这个功能：

In [86]:
f = lambda x: x.max() - x.min()
frame.apply(f)

b    2.277243
d    5.025404
e    2.031723
dtype: float64

* 这里的函数f，计算了一个Series的最大值和最小值的差。

* 如果传递axis='columns'到apply函数，函数将会被每行调用一次：

In [87]:
frame.apply(f, axis='columns')

Utah      1.899503
Ohio      2.457622
Texas     1.922590
Oregon    3.626066
dtype: float64

* 传递给apply的函数不是必须返回一个标量值，还可以返回由多个值组成的Series：

In [ ]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

In [ ]:
format = lambda x: '%.2f' % x
frame.applymap(format)

In [ ]:
frame['e'].map(format)

### （7）排序和排名

* 根据某些准则对数据集排序是另一个重要的内建操作。如需按行或列索引进行字典型排序，需要使用sort_index方法，它将返回一个已排序的新对象：

In [88]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

* 在DataFrame中，可以在各个轴上按索引排序：

In [89]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])
frame.sort_index()
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


* 数据默认是按升序排序的，但也可以降序排序：

In [90]:
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


In [91]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

In [92]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

In [93]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame
frame.sort_values(by='b')

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


In [94]:
frame.sort_values(by=['a', 'b'])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


In [95]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

In [96]:
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

In [97]:
# 将值分配给组中的最大排名
obj.rank(ascending=False, method='max')

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

* DataFrame可以对行或列计算排名：

In [98]:
frame = pd.DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                      'c': [-2, 5, 8, -2.5]})
frame
frame.rank(axis='columns')

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


### （8）含有重复标签的轴索引

* 尽管很多pandas函数（比如reindex）需要标签是唯一的，但这并不是强制性的。下面是一个简单的带有重复索引的Series：

In [99]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

In [ ]:
obj.index.is_unique

In [ ]:
obj['a']
obj['c']

In [ ]:
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df
df.loc['b']

## 3. 描述性统计的概述与计算

* pandas对象拥有一组常用的数学和统计方法。它们大部分都属于约简和汇总统计，用于从Series中提取单个值（如sum或mean）或从DataFrame的行或列中提取一个Series。
* 跟对应的NumPy数组方法相比，它们都是基于没有缺失数据的假设而构建的。

In [49]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [50]:
df.sum()

one    9.25
two   -5.80
dtype: float64

In [51]:
df.sum(axis='columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

* 注意： 除非整个切片上都是NA，否则NA值是被自动排除的。
* 可以通过skipna来实现不排除NA

In [52]:
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

In [53]:
df.mean( skipna=False)

one   NaN
two   NaN
dtype: float64

In [54]:
df.idxmax()

one    b
two    d
dtype: object

In [55]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


* 一次产生多个汇总统计：

In [56]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


In [60]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)
obj

0     a
1     a
2     b
3     c
4     a
5     a
6     b
7     c
8     a
9     a
10    b
11    c
12    a
13    a
14    b
15    c
dtype: object

* 非数值型

In [59]:
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

### （1）相关性与协方差

* 有些汇总统计（如相关系数和协方差）是通过参数对计算出来的。我们来看几个DataFrame，它们的数据来自Yahoo!Finance的股票价格和成交量，使用的是pandas-datareader包（可以用conda或pip安装）：

In [62]:
conda install pandas-datareader

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [65]:
import pandas_datareader.data as web
all_data = {ticker: web.get_data_stooq(ticker)
            for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}
all_data

{'AAPL':                Open     High      Low    Close     Volume
 Date                                                     
 2021-04-16  134.300  134.670  133.280  134.160   84922386
 2021-04-15  133.820  135.000  133.640  134.500   89347102
 2021-04-14  134.940  135.000  131.655  132.030   87222782
 2021-04-13  132.440  134.660  131.930  134.430   91266545
 2021-04-12  132.520  132.850  130.630  131.240   91419983
 ...             ...      ...      ...      ...        ...
 2016-04-26   24.162   24.488   24.162   24.263  169812530
 2016-04-25   24.414   24.567   24.302   24.433  114611235
 2016-04-22   24.416   24.760   24.329   24.574  140709697
 2016-04-21   24.862   24.862   24.539   24.637  125892175
 2016-04-20   24.798   25.137   24.660   24.912  113577434
 
 [1257 rows x 5 columns],
 'IBM':                Open    High     Low   Close   Volume
 Date                                                
 2021-04-16  133.000  134.10  132.95  133.59  5291756
 2021-04-15  133.280  133.87

In [68]:
price = pd.DataFrame({ticker: data['Close']
                     for ticker, data in all_data.items()})

price


,AAPL,IBM,MSFT,GOOG
Date,,,,
2021-04-16,134.160,133.59,260.740,2297.76
2021-04-15,134.500,132.58,259.500,2296.66
2021-04-14,132.030,132.63,255.590,2254.84
2021-04-13,134.430,131.18,258.490,2267.27
2021-04-12,131.240,134.59,255.910,2254.79
...,...,...,...,...
2016-04-26,24.263,119.54,47.168,708.14
2016-04-25,24.433,119.32,47.782,723.15
2016-04-22,24.574,119.10,47.480,718.77


In [70]:
volume = pd.DataFrame({ticker: data['Volume']
                      for ticker, data in all_data.items()})

volume

,AAPL,IBM,MSFT,GOOG
Date,,,,
2021-04-16,84922386,5291756,24878582,1130090
2021-04-15,89347102,3883955,25627481,1373612
2021-04-14,87222782,5868049,23070938,1010967
2021-04-13,91266545,8033530,23837469,1165804
2021-04-12,91419983,3753959,27148668,1565851
...,...,...,...,...
2016-04-26,169812530,3715386,35527728,2010224
2016-04-25,114611235,3549964,34035163,1312381
2016-04-22,140709697,6474083,132676375,5467206


In [71]:
returns = price.pct_change()
returns.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2016-04-26,0.066787,-0.009364,0.009784,0.003259
2016-04-25,0.007007,-0.001840,0.013017,0.021196
2016-04-22,0.005771,-0.001844,-0.006320,-0.006057
2016-04-21,0.002564,0.005374,0.077275,0.056165
2016-04-20,0.011162,-0.021630,-0.003421,-0.008523


* 计算相关性

In [72]:
returns['MSFT'].corr(returns['IBM'])

0.5367670359721808

* 计算协方差

In [73]:
returns['MSFT'].cov(returns['IBM'])

0.0001549849638675364

In [74]:
returns.MSFT.corr(returns.IBM)

0.5367670359721808

* 相关性矩阵

In [76]:
returns.corr()

,AAPL,IBM,MSFT,GOOG
AAPL,1.000000,0.451315,0.720740,0.661110
IBM,0.451315,1.000000,0.536767,0.502356
MSFT,0.720740,0.536767,1.000000,0.784254
GOOG,0.661110,0.502356,0.784254,1.000000


* 协方差矩阵

In [75]:
returns.cov()

,AAPL,IBM,MSFT,GOOG
AAPL,0.000369,0.000143,0.000242,0.000215
IBM,0.000143,0.000273,0.000155,0.000140
MSFT,0.000242,0.000155,0.000305,0.000232
GOOG,0.000215,0.000140,0.000232,0.000286


In [77]:
returns.corrwith(returns.IBM)

AAPL    0.451315
IBM     1.000000
MSFT    0.536767
GOOG    0.502356
dtype: float64

In [78]:
returns.corrwith(volume)

AAPL    0.008208
IBM     0.061590
MSFT    0.008962
GOOG    0.027467
dtype: float64

### （2）唯一值、计数和成员属性

* 还有一类相关的方法可以从一维Series包含的数值中提取信息。看下面的例子：

In [36]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

* unique：它可以得到Series中的唯一值

In [37]:
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

In [38]:
uniques.sort()
uniques

array(['a', 'b', 'c', 'd'], dtype=object)

* value_counts：计算Series包含值的个数

In [39]:
obj.value_counts()

a    3
c    3
b    2
d    1
dtype: int64

In [41]:
pd.value_counts(obj.values, sort=True)

a    3
c    3
b    2
d    1
dtype: int64

* isin: 向量化的成员属性检查

In [42]:
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [44]:
mask = obj.isin(['b', 'c'])
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [45]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

* get_indexer: 提供索引数组

In [46]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c', 'b', 'a'])
pd.Index(unique_vals).get_indexer(to_match)

array([0, 2, 1, 1, 0, 2], dtype=int64)

* 有时，你可能想要计算DataFrame中多个相关列的直方图。例如：

In [47]:
data = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
                     'Qu2': [2, 3, 1, 2, 3],
                     'Qu3': [1, 5, 2, 4, 4]})
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


* 将pandas.value_counts传入该DataFrame的apply函数可以得到：

In [48]:
result = data.apply(pd.value_counts).fillna(0)
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0
